### Linear and Multilinear Functions

In [4]:
from kingdon import Algebra
from gc_utils import *
import numpy as np
# use vanilla algebra, so we're not bother by the sign of I*I.reverse()
alg = Algebra(4) 
locals().update(alg.blades)

In [5]:
#1.1 linear transformation
A = create_random_multivector(alg)
B = create_random_multivector(alg)
C = create_random_multivector(alg)
D = create_random_multivector(alg)
x = create_random_multivector(alg).grade(1)
a = create_random_multivector(alg).grade(1)
f = lambda x: (A*x*B+C*x*D).grade(1)
differential(f, x, a), f(a)

(1.03 𝐞₁ + 4.79 𝐞₂ + 4.83 𝐞₃ + -2.36 𝐞₄,
 1.03 𝐞₁ + 4.79 𝐞₂ + 4.83 𝐞₃ + -2.36 𝐞₄)

In [6]:
#1.2
g = lambda x: (C*x*D).grade(1)
g(f(a)), differential(g, f(a), f(a))

(3.24 𝐞₁ + 0.324 𝐞₂ + -14.4 𝐞₃ + 0.134 𝐞₄,
 3.24 𝐞₁ + 0.324 𝐞₂ + -14.4 𝐞₃ + 0.134 𝐞₄)

In [7]:
# modify the input of the original skew_symmetrizer: a multivector Ar instead of vectors
def skew_symmetrizer(F, Ar, alg, h=1e-6, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(alg.frame)
    drF = 0
    r = Ar.grades[0]
    for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
        # if F is linear, vectors cause no difference in vectors_partial. The input is just Ar.
        # So actually we allow Ar being any multivector
        drF += (Ar.sp(wedge(base_vectors[::-1]))) * vectors_partial(F, np.zeros(r), reci_vectors, h=h)
    return (1/factorial(r)) * drF

In [8]:
#1.4 Extend a vector function to a multivector function.
# For linear f we ignore the position where f is measured, 
# just the differential at each direction.
def outermorphism(f, A: MultiVector, alg, h=1e-6, frame=None, r_frame=None):
    wf = lambda vectors: wedge([f(v) for v in vectors])
    outer = 0
    for r in A.grades:
        if r == 0:
           outer += A.grade(0)
           continue
        outer += skew_symmetrizer(wf, A.grade(r), alg, h, frame, r_frame)
    return outer

outermorphism(f, x, alg), f(x)

(2.18 𝐞₁ + 2.33 𝐞₂ + 4.44 𝐞₃ + -1.54 𝐞₄,
 2.18 𝐞₁ + 2.33 𝐞₂ + 4.44 𝐞₃ + -1.54 𝐞₄)

In [9]:
#1.6 A frame can be arbitrary as long as it represent the same space. 
# It's pretty handy to specify a domain.
h = 1e-2
A2 = e12
A2frame = [e1, e2]
A2r_frame = reciprocal(A2frame)

outermorphism(f, P(A, A2), alg, h, frame=A2frame, r_frame=A2r_frame), outermorphism(f, A, alg, h, frame=A2frame, r_frame=A2r_frame)

(0.163 + -1.47 𝐞₁ + 0.904 𝐞₂ + 3.0 𝐞₃ + -3.11 𝐞₄ + 1.05 𝐞₁₂ + -13.8 𝐞₁₃ + 13.0 𝐞₁₄ + 10.6 𝐞₂₃ + -10.2 𝐞₂₄ + 2.74 𝐞₃₄,
 0.163 + -1.47 𝐞₁ + 0.904 𝐞₂ + 3.0 𝐞₃ + -3.11 𝐞₄ + 1.05 𝐞₁₂ + -13.8 𝐞₁₃ + 13.0 𝐞₁₄ + 10.6 𝐞₂₃ + -10.2 𝐞₂₄ + 2.74 𝐞₃₄)

In [10]:
#1.7 An outermorphism preserves outer products
vectors = create_r_vectors(3, alg)
outermorphism(f, wedge(vectors), alg), wedge([f(v) for v in vectors])

(16.0 𝐞₁₂₃ + -9.3 𝐞₁₂₄ + -1.91 𝐞₁₃₄ + 4.94 𝐞₂₃₄,
 16.0 𝐞₁₂₃ + -9.3 𝐞₁₂₄ + -1.91 𝐞₁₃₄ + 4.94 𝐞₂₃₄)

In [11]:
# By 1.7, splitting A into simple blades, we get an easier implement of 1.4
# Here, the value at v replaces the slope at direction v by linearity
def outermorphism_(f, A: MultiVector, alg):
    outer = A.grade(0)
    for vectors in multi_frame_vectors(alg.frame):
        Vr = wedge(reciprocal(vectors)[::-1])
        outer += alg.sp(Vr,A) * wedge([f(v) for v in vectors]) # frame split of A
    return outer

outermorphism_(f, D, alg), outermorphism(f, D, alg)

(0.793 + 0.839 𝐞₁ + 7.59 𝐞₂ + 1.41 𝐞₃ + 0.515 𝐞₄ + -7.88 𝐞₁₂ + 0.00175 𝐞₁₃ + 0.0855 𝐞₁₄ + -14.9 𝐞₂₃ + 20.4 𝐞₂₄ + 1.4 𝐞₃₄ + 1.91e+02 𝐞₁₂₃ + -1.21e+02 𝐞₁₂₄ + -17.3 𝐞₁₃₄ + 27.8 𝐞₂₃₄ + 66.4 𝐞₁₂₃₄,
 0.793 + 0.839 𝐞₁ + 7.59 𝐞₂ + 1.41 𝐞₃ + 0.515 𝐞₄ + -7.88 𝐞₁₂ + 0.00175 𝐞₁₃ + 0.0855 𝐞₁₄ + -14.9 𝐞₂₃ + 20.4 𝐞₂₄ + 1.4 𝐞₃₄ + 1.91e+02 𝐞₁₂₃ + -1.21e+02 𝐞₁₂₄ + -17.3 𝐞₁₃₄ + 27.8 𝐞₂₃₄ + 66.4 𝐞₁₂₃₄)

In [12]:
#1.9 anoter outer product property
outermorphism(f, A.reverse(), alg), outermorphism(f, A, alg).reverse()

(0.163 + -1.31 𝐞₁ + 7.45 𝐞₂ + 4.21 𝐞₃ + -2.64 𝐞₄ + -23.8 𝐞₁₂ + 5.41 𝐞₁₃ + -12.8 𝐞₁₄ + 0.46 𝐞₂₃ + 4.75 𝐞₂₄ + -4.6 𝐞₃₄ + -1.32e+02 𝐞₁₂₃ + 99.1 𝐞₁₂₄ + 36.3 𝐞₁₃₄ + -19.4 𝐞₂₃₄ + 1.42e+02 𝐞₁₂₃₄,
 0.163 + -1.31 𝐞₁ + 7.45 𝐞₂ + 4.21 𝐞₃ + -2.64 𝐞₄ + -23.8 𝐞₁₂ + 5.41 𝐞₁₃ + -12.8 𝐞₁₄ + 0.46 𝐞₂₃ + 4.75 𝐞₂₄ + -4.6 𝐞₃₄ + -1.32e+02 𝐞₁₂₃ + 99.1 𝐞₁₂₄ + 36.3 𝐞₁₃₄ + -19.4 𝐞₂₃₄ + 1.42e+02 𝐞₁₂₃₄)

In [13]:
#1.10
fA2 = lambda x: P(f(x), A2)
outermorphism(fA2, A, alg), P(outermorphism(fA2, A, alg), A2)

(0.163 + -1.31 𝐞₁ + 7.45 𝐞₂ + 23.8 𝐞₁₂, 0.163 + -1.31 𝐞₁ + 7.45 𝐞₂ + 23.8 𝐞₁₂)

In [14]:
#1.11
def adjoint_outermorphism(f, A, alg, h=1e-6, frame=None, r_frame=None):
    F = lambda vectors: wedge([f(v) for v in vectors]).sp(A)
    outer = 0
    for r in A.grades:
        if r == 0:
           outer += A.grade(0)
           continue
        # Why np.zeros(r)? The derivative of a linear function is constant
        outer += simplicial_derivative(F, np.zeros(r), alg, h, frame, r_frame)
    return outer

In [15]:
#1.12
adjoint_outermorphism(fA2, P(A, A2), alg), adjoint_outermorphism(fA2, A, alg)

(0.163 + -1.97 𝐞₁ + 2.14 𝐞₂ + 2.33 𝐞₃ + 1.49 𝐞₄ + 1.05 𝐞₁₂ + -15.9 𝐞₁₃ + -15.2 𝐞₁₄ + 18.5 𝐞₂₃ + 17.3 𝐞₂₄ + 5.97 𝐞₃₄,
 0.163 + -1.97 𝐞₁ + 2.14 𝐞₂ + 2.33 𝐞₃ + 1.49 𝐞₄ + 1.05 𝐞₁₂ + -15.9 𝐞₁₃ + -15.2 𝐞₁₄ + 18.5 𝐞₂₃ + 17.3 𝐞₂₄ + 5.97 𝐞₃₄)

In [16]:
#1.13
# Exercise: implement the adjoint outermorphism by spliting A into vectors.
adjoint_outermorphism(f, A^B, alg), adjoint_outermorphism(f, A, alg) ^ adjoint_outermorphism(f, B, alg)

(0.12 + -1.36 𝐞₁ + 1.79 𝐞₂ + 1.94 𝐞₃ + 3.55 𝐞₄ + -3.57 𝐞₁₂ + -17.7 𝐞₁₃ + -23.7 𝐞₁₄ + 21.9 𝐞₂₃ + 30.2 𝐞₂₄ + 11.9 𝐞₃₄ + 45.8 𝐞₁₂₃ + 52.1 𝐞₁₂₄ + -1.42e+02 𝐞₁₃₄ + 1.31e+02 𝐞₂₃₄ + 2.29e+02 𝐞₁₂₃₄,
 0.12 + -1.36 𝐞₁ + 1.79 𝐞₂ + 1.94 𝐞₃ + 3.55 𝐞₄ + -3.57 𝐞₁₂ + -17.7 𝐞₁₃ + -23.7 𝐞₁₄ + 21.9 𝐞₂₃ + 30.2 𝐞₂₄ + 11.9 𝐞₃₄ + 45.8 𝐞₁₂₃ + 52.1 𝐞₁₂₄ + -1.42e+02 𝐞₁₃₄ + 1.31e+02 𝐞₂₃₄ + 2.29e+02 𝐞₁₂₃₄)

In [17]:
#1.14a r <= s
r, s = 2, 3
Ar = A.grade(r)
Bs = B.grade(s)
(
    Ar | adjoint_outermorphism(f, Bs, alg),
    outermorphism(f, Ar, alg) | Bs,
    adjoint_outermorphism(f, outermorphism(f, Ar, alg) | Bs, alg),
)

(26.5 𝐞₁ + -10.3 𝐞₂ + 23.9 𝐞₃ + 6.62 𝐞₄,
 2.9 𝐞₁ + 5.63 𝐞₂ + -8.88 𝐞₃ + -14.6 𝐞₄,
 26.5 𝐞₁ + -10.3 𝐞₂ + 23.9 𝐞₃ + 6.62 𝐞₄)

In [18]:
#1.14b r >= s
r, s = 3, 2
Ar = A.grade(r)
Bs = B.grade(s)
(
    outermorphism(f, Ar | adjoint_outermorphism(f, Bs, alg), alg),
    Ar | adjoint_outermorphism(f, Bs, alg),
    outermorphism(f, Ar, alg) | Bs,
)

(-40.6 𝐞₁ + -9.84 𝐞₂ + -53.6 𝐞₃ + 31.6 𝐞₄,
 -8.11 𝐞₁ + -18.1 𝐞₂ + -4.19 𝐞₃ + -4.51 𝐞₄,
 -40.6 𝐞₁ + -9.84 𝐞₂ + -53.6 𝐞₃ + 31.6 𝐞₄)

In [19]:
#1.15 r = s
for r,s in [(1,1), (2,2), (3,3)]:
    Ar = A.grade(r)
    Bs = B.grade(s)
    print(
        Ar | adjoint_outermorphism(f, Bs, alg),
        outermorphism(f, Ar, alg) | Bs,
    )

7.62 7.62
-12.9 -12.9
11.4 11.4


In [20]:
# proof of 1.14a
r, s = 2, 3
Ar = A.grade(r)
Bs = B.grade(s)
frame = alg.frame
r_frame = reciprocal(alg.frame)
rhs = 0
r = Ar.grades[0]
F = lambda vectors: wedge([f(v) for v in vectors]) 
for base_vectors, reci_vectors in zip(permutations(frame, s), permutations(r_frame, s)):
    rhs += (Ar | (wedge(base_vectors[::-1]))) * (vectors_partial(F, np.zeros(s), reci_vectors, h=h) | Bs)
(
    Ar | adjoint_outermorphism(f, Bs, alg),
    rhs / factorial(s)
)

(26.5 𝐞₁ + -10.3 𝐞₂ + 23.9 𝐞₃ + 6.62 𝐞₄,
 26.5 𝐞₁ + -10.3 𝐞₂ + 23.9 𝐞₃ + 6.62 𝐞₄)

In [21]:
# singular, nonsingular functions
I = alg.pseudoscalar((1,))
# fA2 is singular
outermorphism(fA2, I, alg), outermorphism(f, I, alg), adjoint_outermorphism(f, I, alg)

(, 2.51e+02 𝐞₁₂₃₄, 2.51e+02 𝐞₁₂₃₄)

In [22]:
#1.16 Use inverse instead of reverse for negative signatures. case I = Ip
n = 4
Ir = I.inv()
F = lambda vectors: wedge([f(v) for v in vectors])
(
    simplicial_derivative(F, alg.frame, alg, h=1e-2), 
    Ir * outermorphism(f, I, alg), 
    adjoint_outermorphism(f, I, alg) * Ir
)

(2.51e+02, 2.51e+02, 2.51e+02)

In [23]:
#1.16 case I != Ip
n = 2
I = e12
Ip = e23
Ir = I.inv()
Ipr = Ip.inv()
h = 1e-2
# a function from I to Ip
g = lambda x: P((A*P(x, I)*B).grade(1), Ip)
F = lambda vectors: wedge([g(v) for v in vectors])
(
    simplicial_derivative(F, alg.frame[:2], alg, h),
    Ir * outermorphism(g, I, alg, h),
    adjoint_outermorphism(g, Ip, alg, h) * Ipr
)

(-2.87 𝐞₁₃, -2.87 𝐞₁₃, -2.87 𝐞₁₃)

In [24]:
#1.17a
outermorphism(g, I, alg, h), Ip

(2.87 𝐞₂₃, 1 𝐞₂₃)

In [25]:
#1.17b Need extra minus sign for negative signature. You can try Algebra(3,1).
adjoint_outermorphism(g, Ip, alg, h), I

(2.87 𝐞₁₂, 1 𝐞₁₂)

In [26]:
# The relative orientation between I and I' can be arbitrary.
# Here we let the parent algebra determine the orientation.
def det(f, I, Ip):
    return (outermorphism(f, I, alg)/Ip)[0]

det(g, I, Ip)

2.874148196910696

In [27]:
#1.21a null part goes null
C = create_random_multivector(alg)
C_ = outermorphism(g, C, alg)
(1/det(g, I, Ip)) * adjoint_outermorphism(g, C_*Ip, alg) * Ir, C

(0.345 + 0.325 𝐞₁ + 0.197 𝐞₂ + 0.224 𝐞₁₂,
 0.345 + 0.325 𝐞₁ + 0.197 𝐞₂ + 0.299 𝐞₃ + 0.818 𝐞₄ + 0.224 𝐞₁₂ + 0.669 𝐞₁₃ + 0.885 𝐞₁₄ + 0.858 𝐞₂₃ + 0.641 𝐞₂₄ + 0.576 𝐞₃₄ + 0.829 𝐞₁₂₃ + 0.156 𝐞₁₂₄ + 0.349 𝐞₁₃₄ + 0.867 𝐞₂₃₄ + 0.909 𝐞₁₂₃₄)

In [28]:
#1.21b null part goes null
C_ = create_random_multivector(alg)
C = adjoint_outermorphism(g, C_, alg)
(1/det(g, I, Ip)) * outermorphism(g, C*I, alg) * Ipr, C_

(0.118 + 0.371 𝐞₂ + 0.45 𝐞₃ + 0.562 𝐞₂₃,
 0.118 + 0.89 𝐞₁ + 0.371 𝐞₂ + 0.45 𝐞₃ + 0.868 𝐞₄ + 0.195 𝐞₁₂ + 0.768 𝐞₁₃ + 0.621 𝐞₁₄ + 0.562 𝐞₂₃ + 0.167 𝐞₂₄ + 0.207 𝐞₃₄ + 0.371 𝐞₁₂₃ + 0.381 𝐞₁₂₄ + 0.111 𝐞₁₃₄ + 0.994 𝐞₂₃₄ + 0.401 𝐞₁₂₃₄)

In [29]:
#2.1 Note that characteristic multivectors are of even grades
r = 3  # try several r
h = 1e-2
vectors = create_r_vectors(r, alg)
F = lambda vectors: wedge([f(v) for v in vectors])
(
    F(vectors),
    outermorphism(f, wedge(vectors), alg, h),
    simplicial_derivative(F, vectors, alg, h), # characteristic multivectors
    derivative(lambda A: outermorphism(f, A, alg, h), wedge(vectors), alg, h, r),
)


(-2.71 𝐞₁₂₃ + -0.429 𝐞₁₂₄ + -1.29 𝐞₁₃₄ + 2.09 𝐞₂₃₄,
 -2.71 𝐞₁₂₃ + -0.429 𝐞₁₂₄ + -1.29 𝐞₁₃₄ + 2.09 𝐞₂₃₄,
 63.3 + 0.142 𝐞₁₂ + -42.9 𝐞₁₃ + -46.9 𝐞₁₄ + -3.13 𝐞₂₃ + -19.7 𝐞₂₄ + -71.5 𝐞₃₄,
 63.3 + 0.142 𝐞₁₂ + -42.9 𝐞₁₃ + -46.9 𝐞₁₄ + -3.13 𝐞₂₃ + -19.7 𝐞₂₄ + -71.5 𝐞₃₄)

In [30]:
#2.2
frame = create_r_vectors(len(alg.signature), alg)
r_frame = reciprocal(frame)
images = [f(v) for v in frame]
(
    sum(vr * b for b, vr in zip(images, r_frame)),
    derivative(f, x, alg)
)

(-4.34 + -4.4 𝐞₁₂ + 1.71 𝐞₁₃ + -0.754 𝐞₁₄ + -1.73 𝐞₂₃ + -3.86 𝐞₂₄ + -2.12 𝐞₃₄,
 -4.34 + -7.53 𝐞₁ + 0.225 𝐞₂ + 1.86 𝐞₃ + 2.47 𝐞₄ + 2.13 𝐞₁₂ + 3.88 𝐞₁₃ + 2.63 𝐞₁₄ + 0.552 𝐞₂₃ + -3.16 𝐞₂₄ + 0.315 𝐞₃₄ + 1.36 𝐞₁₂₃ + -0.177 𝐞₁₂₄ + 0.181 𝐞₁₃₄ + 2.03 𝐞₂₃₄ + 1.5 𝐞₁₂₃₄)

In [31]:
#2.2 comes from linearity
images, [differential(f, a, v) for v in frame]

([3.47 𝐞₁ + 5.5 𝐞₂ + 4.2 𝐞₃ + -0.722 𝐞₄,
  2.39 𝐞₁ + 6.23 𝐞₂ + 5.69 𝐞₃ + -1.5 𝐞₄,
  2.31 𝐞₁ + 3.29 𝐞₂ + 4.57 𝐞₃ + -1.19 𝐞₄,
  1.88 𝐞₁ + 4.87 𝐞₂ + 1.8 𝐞₃ + -0.277 𝐞₄],
 [3.47 𝐞₁ + 5.5 𝐞₂ + 4.2 𝐞₃ + -0.722 𝐞₄,
  2.39 𝐞₁ + 6.23 𝐞₂ + 5.69 𝐞₃ + -1.5 𝐞₄,
  2.31 𝐞₁ + 3.29 𝐞₂ + 4.57 𝐞₃ + -1.19 𝐞₄,
  1.88 𝐞₁ + 4.87 𝐞₂ + 1.8 𝐞₃ + -0.277 𝐞₄])

In [32]:
#2.3 trace
def trace(f, alg):
    return sum(vr.sp(f(v)) for v, vr in zip(alg.frame, reciprocal(alg.frame)))
def divergence(f, alg):
    return sum(vr | f(v) for v, vr in zip(alg.frame, reciprocal(alg.frame)))
(
    derivative(f, e1, alg).grade(0),
    trace(f, alg),
    divergence(f, alg)
)

(-4.34, -4.34, -4.34)

In [33]:
#2.4
r = 2
h = 1e-2
F = lambda vectors: wedge([f(v) for v in vectors])
def char_multi_linear(f, r, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(frame)
    r_vec_frame = r_vector_frame_vectors(frame, r)
    r_reci_frame = r_vector_frame_vectors(r_frame, r, reverse=True)
    sdf = 0
    for vectors, r_vectors in zip(r_vec_frame, r_reci_frame):
        sdf += wedge(r_vectors[::-1]) * wedge([f(v) for v in vectors])
    return sdf

(
    char_multi_linear(f, r, alg),
    simplicial_derivative(F, np.zeros(r), alg, h)
)

(-7.94 + 4.32 𝐞₁₂ + 18.3 𝐞₁₃ + 19.5 𝐞₁₄ + -2.4 𝐞₂₃ + 22.9 𝐞₂₄ + 14.8 𝐞₃₄ + 17.2 𝐞₁₂₃₄,
 -7.94 + 4.32 𝐞₁₂ + 18.3 𝐞₁₃ + 19.5 𝐞₁₄ + -2.4 𝐞₂₃ + 22.9 𝐞₂₄ + 14.8 𝐞₃₄ + 17.2 𝐞₁₂₃₄)

In [34]:
#2.6 when full rank, only scalar remains
def trans2matrix(f, alg):
    values = [f(a) for a in alg.frame]
    return np.array([[ar.sp(v)[0] for v in values] for ar in reciprocal(alg.frame)])
matrix = trans2matrix(f, alg)
I = alg.pseudoscalar((1,))
char_multi_linear(f, 4, alg)[0], np.linalg.det(matrix), det(f, I, I)

(-41.25393895119303, -41.253938951193014, -41.25393895119305)

In [35]:
#2.7 sum
r = 3
h = lambda x: f(x)+g(x)

# Sometimes I use a random frame to check frame independence
frame = create_r_vectors(len(alg.signature), alg)
r_frame = reciprocal(frame)
drF = 0
coef = [1/(factorial(r-s)*factorial(s)) for s in range(r+1)]
for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
    gh = 0
    for s in range(r+1):
        gh += coef[s]*wedge([f(v) for v in reci_vectors[:s]]) ^ wedge([g(v) for v in reci_vectors[s:]])
    drF += wedge(base_vectors[::-1]) * gh
char_multi_linear(h, r, alg), drF

(1.63e+02 + -0.356 𝐞₁₂ + -70.3 𝐞₁₃ + -1.15e+02 𝐞₁₄ + -3.24 𝐞₂₃ + -49.5 𝐞₂₄ + -1.61e+02 𝐞₃₄,
 1.63e+02 + -0.356 𝐞₁₂ + -70.3 𝐞₁₃ + -1.15e+02 𝐞₁₄ + -3.24 𝐞₂₃ + -49.5 𝐞₂₄ + -1.61e+02 𝐞₃₄)

In [36]:
#2.8 characteristic polynomial
def char_scalar(f, r, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(frame)
    r_vec_frame = r_vector_frame_vectors(frame, r)
    r_reci_frame = r_vector_frame_vectors(r_frame, r, reverse=True)
    sdf = 0
    for vectors, r_vectors in zip(r_vec_frame, r_reci_frame):
        sdf += alg.sp(wedge(r_vectors[::-1]), wedge([f(v) for v in vectors]))[0]
    return sdf

# This calculates n characteristic multivectors. 
# But drops the nonscalar information
def char_poly_coefs(f, alg, frame=None, r_frame=None):
    n = alg.d
    if frame:
        n = len(frame)
    return [(-1)**(n-s) * char_scalar(f, s, alg, frame, r_frame) for s in range(n+1)]

def polynomial(l, coefs):
    return sum(l ** i * coef for i, coef in enumerate(coefs[::-1]))

l = 3.3
n = 4
(
    char_multi_linear(lambda x: f(x) - l*x, n, alg), 
    polynomial(l, char_poly_coefs(f, alg))
)

(-62.1, -62.05607735165229)

In [37]:
def r_fold(f,r,value):
    return reduce(lambda v, _: f(v), range(r), value)

r_fold(f, 3, x), f(f(f(x))), r_fold(f, 0, x), x

(50.9 𝐞₁ + 33.6 𝐞₂ + 46.3 𝐞₃ + -0.475 𝐞₄,
 50.9 𝐞₁ + 33.6 𝐞₂ + 46.3 𝐞₃ + -0.475 𝐞₄,
 0.187 𝐞₁ + 0.86 𝐞₂ + 0.0579 𝐞₃ + 0.769 𝐞₄,
 0.187 𝐞₁ + 0.86 𝐞₂ + 0.0579 𝐞₃ + 0.769 𝐞₄)

In [38]:
#2.9 Cayley Hamilton
sum(r_fold(f, i, x) * coef for i, coef in enumerate(char_poly_coefs(f, alg)[::-1]))

7.11e-14 𝐞₁ + -2.84e-14 𝐞₂ + -1.42e-14 𝐞₄

In [39]:
#2.10 We can find eigenvalues like the usual many-fold method.
# How about eigenvectors? 
# What else can we get from characteristic multivector? Some papers I should go through...
matrix = trans2matrix(f, alg)
eigvalues, eigvecs = np.linalg.eig(matrix)
coefs = char_poly_coefs(f, alg)
eigvalues, np.roots(coefs)

(array([ 3.60663547+0.j        , -0.74429743+0.j        ,
        -3.60336388+1.54393778j, -3.60336388-1.54393778j]),
 array([ 3.60663547+0.j        , -3.60336388+1.54393778j,
        -3.60336388-1.54393778j, -0.74429743+0.j        ]))

In [40]:
# From Lasenby's paper, we can reconstruct a rotor from characteristic multivector:
# Note that the rotor must have non-zero scalar part
# the paper suggest such method is pretty robust for noisy data
R = blade_exp(create_r_blade(2, alg)) * blade_exp(create_r_blade(2, alg))
rotor = lambda x: R.sw(x) # a screw motion

terms_ratio(R.reverse(), sum(char_multi_linear(rotor, s, alg) for s in range(n+1)))

array([0.21313751, 0.21313751, 0.21313751, 0.21313751, 0.21313751,
       0.21313751, 0.21313751, 0.21313751])

In [41]:
# recovering matrix to transfromation
def matrix2trans(M, alg:Algebra ):
    d = alg.d
    assert M.shape[0] == d, 'dimension not fit'
    return lambda x: sum(c*e for c,e in zip(np.dot(M, x.asfullmv()[1:d+1]), alg.frame))
    
mf = matrix2trans(matrix, alg)
mf(x), f(x)

(3.43 𝐞₁ + 2.92 𝐞₂ + 4.42 𝐞₃ + -0.931 𝐞₄,
 3.43 𝐞₁ + 2.92 𝐞₂ + 4.42 𝐞₃ + -0.931 𝐞₄)

In [42]:
#2.11
s = 2
rhs = (1/s) * sum((-1)**(r+1) * char_scalar(f, s-r, alg) * trace(lambda x: r_fold(f, r, x), alg)[0] for r in range(1,s+1))
char_scalar(f, s, alg), rhs

(-7.944525415130959, -7.944525415130952)

In [43]:
#2.13
k = 3
char_scalar(f, k, alg), sum(np.prod(comb) for comb in combinations(eigvalues, k))

(63.33414856061758, (63.334148560617635+0j))

In [44]:
#2.14 It's easier to show the eigenvalues of f^k are {lambda^k_i} by linearity
trace(lambda x: r_fold(f, k, x), alg)[0], sum(l**k for l in eigvalues)

(4.46529747934181, (4.465297479341892+0j))

In [45]:
# 3.4 A space must bound f to own an invariant subspace.
I = alg.pseudoscalar((1,))
outermorphism(f, I, alg), det(f, I, I)*I, adjoint_outermorphism(f, I, alg)

(-41.3 𝐞₁₂₃₄, -41.3 𝐞₁₂₃₄, -41.3 𝐞₁₂₃₄)

In [46]:
#3.6
l = 2.2
n = 4
F = lambda x: f(x) - l*x
coefs = char_poly_coefs(f, alg)
outermorphism(F, I, alg), polynomial(l, coefs) * I

(-1.49e+02 𝐞₁₂₃₄, -1.49e+02 𝐞₁₂₃₄)

In [47]:
#3.7
I = e12
frame = [e1,e2]
g = lambda x: P((A*x.grade(1)*B).grade(1), I)
coefs = char_poly_coefs(g, alg, frame, frame)
F = lambda x: g(x) - l*x
outermorphism(F, I, alg), polynomial(l, coefs) * I

(8.87 𝐞₁₂, 8.87 𝐞₁₂)

In [48]:
#3.8
I = e123
frame2 = [e3]
frame3 = [e1,e2,e3]
h = lambda x: P((A*P(x, e12).grade(1)*B).grade(1), e12) + P((B*P(x, e3).grade(1)*C).grade(1), e3)
F = lambda x: h(x) - l*x
(
    char_scalar(F, 2, alg, frame, frame) * char_scalar(F, 1, alg, frame2, frame2),
    char_scalar(F, 3, alg, frame3, frame3)
)

(-26.19244359899903, -26.19244359899903)

In [49]:
outermorphism(h, e1, alg), adjoint_outermorphism(h, e1, alg)

(-1.93 𝐞₁ + 1.19 𝐞₂, -1.93 𝐞₁ + 3.04 𝐞₂)

In [50]:
#3.9a
adjoint_outermorphism(h, e123, alg) | e12, outermorphism(h, e12, alg).e12 * outermorphism(h, -e3, alg)

(-1.53 𝐞₃, -1.53 𝐞₃)

In [51]:
#3.9b
outermorphism(h, e123, alg) | e3, adjoint_outermorphism(h, e3, alg).e3 * adjoint_outermorphism(h, e12, alg)

(1.53 𝐞₁₂, 1.53 𝐞₁₂)

In [52]:
def separate_indices(numbers):
    real_indices = []
    conjugate_pairs_indices = []
    visited = set()  # To keep track of indices already processed

    for i, num in enumerate(numbers):
        if np.isreal(num):  # Check if the number is real
            real_indices.append(i)
        elif i not in visited:  # Check if the index is not already processed
            conjugate = np.conj(num)
            for j in range(i + 1, len(numbers)):
                if numbers[j] == conjugate:
                    conjugate_pairs_indices.append((i, j))
                    visited.add(i)
                    visited.add(j)
                    break

    return real_indices, conjugate_pairs_indices

separate_indices(eigvalues), eigvalues

(([0, 1], [(2, 3)]),
 array([ 3.60663547+0.j        , -0.74429743+0.j        ,
        -3.60336388+1.54393778j, -3.60336388-1.54393778j]))

In [53]:
# An example of non proper blade:
def linalg_eigblades(f, alg: Algebra):
    matrix = trans2matrix(f, alg)
    eigvalues, eigvecs = np.linalg.eig(matrix)
    real_indices, conjugate_pairs_indices = separate_indices(eigvalues)
    eigblades = [alg.vector(np.real(eigvecs[:, i])) for i in real_indices]
    for k, l in conjugate_pairs_indices:
        v1 = np.real(eigvecs[:,k] + eigvecs[:,l])
        v2 = np.imag(eigvecs[:,k] - eigvecs[:,l])
        eigblades.append(alg.vector(v1) ^ alg.vector(v2))
    return eigblades 

def terms_ratio(A, B: MultiVector):
    valid_keys = [k for k in B.keys() if not np.isclose(B[k], 0)]
    return np.divide([A[k] for k in valid_keys], [B[k] for k in valid_keys])

eigblades = linalg_eigblades(f, alg)

# right eigenblades but not left!
for blade in eigblades:
    print(
        terms_ratio(outermorphism(f, blade, alg), blade), 
        terms_ratio(adjoint_outermorphism(f, blade, alg), blade)
        )

[3.60663547 3.60663547 3.60663547 3.60663547] [   0.48645575    5.84294487    3.98245287 -131.91175015]
[-0.74429743 -0.74429743 -0.74429743 -0.74429743] [ 0.9300258  37.93002006  0.11018615 -4.0444254 ]
[15.36797515 15.36797515 15.36797515 15.36797515 15.36797515 15.36797515] [ 27.33457337  11.20685489 -11.78320951  20.21691102  27.55975988
  38.77824838]


In [54]:
#3.11 reciprocal for blades
def reciprocal(blades):
    I = wedge(blades)
    dualblades = []
    for k in range(len(blades)):
        sign = (-1) ** (blades[k].grades[0] * sum(blade.grades[0] for blade in blades[:k]))
        dualblades.append(sign * wedge(blades[:k] + blades[k+1:]) * I.inv())
    return dualblades

dualblades = reciprocal(eigblades)
# The reciprocal of a right eigenblade is a left eigenblade! Note that they correspond to the same eigenvalues
for blade in dualblades:
    print(
        terms_ratio(outermorphism(f, blade, alg), blade), 
        terms_ratio(adjoint_outermorphism(f, blade, alg), blade)
        )


[23.76461122 10.19663866  4.90608813 -1.03245386] [3.60663547 3.60663547 3.60663547 3.60663547]
[ -0.20032361 -84.30777903   4.2660401   -1.02317671] [-0.74429743 -0.74429743 -0.74429743 -0.74429743]
[29.96770873 14.00328854 -5.10263128 21.98793665 90.20912648  5.57750521] [15.36797515 15.36797515 15.36797515 15.36797515 15.36797515 15.36797515]


In [55]:
#3.12 
[eigblades[i] | dualblades[j] for i,j in [(0,0), (0,1), (1,1)]]

[1.0, 4.86e-17, 1.0]

In [56]:
# What transformation have a non-trivial proper blade?
# A invariant subspace, say e123 of e1234
# from 3.12 we know we're looking for blades equal to thier own reciprocal
# that means orthogonal to other eigenspaces

pf = lambda x: P(f(P(x, e123)), e123) + P(f(P(x, e4)), e4)
[(outermorphism(pf, blade, alg), adjoint_outermorphism(pf, blade, alg)) for blade in [e123, e4]]

[(57.2 𝐞₁₂₃, 57.2 𝐞₁₂₃), (-1.34 𝐞₄, -1.34 𝐞₄)]

In [57]:
eigblades = linalg_eigblades(pf, alg)
for E in [e123, e4]:
    for blade in eigblades: # not just vectors
        blade = (blade | E)*E
        om = outermorphism(pf, blade, alg)
        am = adjoint_outermorphism(pf, blade, alg)
        print(
            terms_ratio(om, blade), 
            terms_ratio(am, blade)
            )

[3.74336823 3.74336823 3.74336823] [0.46402456 5.7160344  4.05901492]
[] []
[15.28234934 15.28234934 15.28234934] [13.72209342 11.98669286 53.33133976]
[] []
[-1.34226765] [-1.34226765]
[] []


In [58]:
#4.1b A matrix from a symmetric transformation is symmetric
fp = lambda x: (f(x) + adjoint_outermorphism(f, x, alg))/2
trans2matrix(fp, alg), fp(x), derivative(lambda x: x | f(x)/2, x, alg, grade=1)

(array([[-1.81931255,  2.33100255,  0.95760899, -0.54688985],
        [ 2.33100255, -0.65888908,  3.527373  ,  2.23419691],
        [ 0.95760899,  3.527373  , -0.52392045,  1.30686478],
        [-0.54688985,  2.23419691,  1.30686478, -1.34226765]]),
 1.3 𝐞₁ + 1.79 𝐞₂ + 4.19 𝐞₃ + 0.863 𝐞₄,
 1.3 𝐞₁ + 1.79 𝐞₂ + 4.19 𝐞₃ + 0.863 𝐞₄)

In [59]:
# a curl is a part of a derivative
derivative(f, x, alg, grade=1).grade(2), curl(f, x, alg, grade=1)

(-4.4 𝐞₁₂ + 1.71 𝐞₁₃ + -0.754 𝐞₁₄ + -1.73 𝐞₂₃ + -3.86 𝐞₂₄ + -2.12 𝐞₃₄,
 -4.4 𝐞₁₂ + 1.71 𝐞₁₃ + -0.754 𝐞₁₄ + -1.73 𝐞₂₃ + -3.86 𝐞₂₄ + -2.12 𝐞₃₄)

In [60]:
#4.1c a skew transfromation leads to a skew matrix
fm = lambda x: (f(x) - adjoint_outermorphism(f, x, alg))/2
trans2matrix(fm, alg), fm(x), x | derivative(f, x, alg, grade=1).grade(2)/2

(array([[ 1.11022302e-16,  2.20122860e+00, -8.55162000e-01,
          3.77072561e-01],
        [-2.20122860e+00, -1.11022302e-16,  8.66388054e-01,
          1.93151556e+00],
        [ 8.55162000e-01, -8.66388054e-01, -5.55111512e-17,
          1.05830612e+00],
        [-3.77072561e-01, -1.93151556e+00, -1.05830612e+00,
          0.00000000e+00]]),
 2.13 𝐞₁ + 1.12 𝐞₂ + 0.229 𝐞₃ + -1.79 𝐞₄,
 2.13 𝐞₁ + 1.12 𝐞₂ + 0.229 𝐞₃ + -1.79 𝐞₄)

In [61]:
#4.2 fp curl free
a | fp(x), x | fp(a), curl(fp, x, alg, grade=1)

(4.53,
 4.53,
 8.15e-10 𝐞₁₃ + 2.33e-10 𝐞₁₄ + 1.16e-10 𝐞₂₃ + -1.16e-10 𝐞₂₄ + 4.07e-10 𝐞₃₄)

In [62]:
#4.3 spectral decomposition of the symmetric part
matrix = trans2matrix(fp, alg)
eigvalues, eigvecs = np.linalg.eigh(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
mf = lambda x: sum(e * P(x, v) for e, v in zip(eigvalues, eigvecs))
mf(x), fp(x)

(1.3 𝐞₁ + 1.79 𝐞₂ + 4.19 𝐞₃ + 0.863 𝐞₄, 1.3 𝐞₁ + 1.79 𝐞₂ + 4.19 𝐞₃ + 0.863 𝐞₄)

In [63]:
# QR factorization
transframe = [fp(e) for e in alg.frame] #another basis
gs_trans = gram_schmidt(transframe)
gs_trans2 = gram_schmidt([fp(v / norm(v)) for v in gs_trans[::-1]])[::-1]
gs_trans2


[-1.61e+03 𝐞₁ + -1.16e-14 𝐞₂ + 4.26e-14 𝐞₃ + 2.99e-13 𝐞₄,
 7.86e-16 𝐞₁ + -1.09e+02 𝐞₂ + -2.57e-14 𝐞₃ + -7e-14 𝐞₄,
 -1.76e-16 𝐞₁ + 1.57e-15 𝐞₂ + -6.64 𝐞₃ + -5.35e-16 𝐞₄,
 -2.57e-16 𝐞₁ + 8.88e-16 𝐞₂ + 1.11e-16 𝐞₃ + -1.38 𝐞₄]

In [64]:
# A GA version of eigenblade-solver given the eigenvalues
def null_space(f, alg, tol=1e-5, frame=None):
    if not frame:
        frame = alg.frame
    null_sp = wedge(frame)
    images = [f(a) for a in frame]
    for v in images:
        contracted = v | null_sp
        if normsq(contracted) >= tol:
            null_sp = contracted
    return null_sp

def eigenblades_from_values(f, eigvalues, alg, frame=None):
    eigblades = []
    for e in eigvalues:
        null_f = lambda x: f(x) - e*x
        eigblades.append(null_space(null_f, alg, frame=frame))
    return eigblades

In [65]:
# One thing to note for a symmetric transformation:
# the characteristic multivectors are of scalar values
# So all subspaces are invariant under the transfromation.
[char_multi_linear(fp, r, alg) for r in range(5)]

[1,
 -4.34 + -8.88e-16 𝐞₁₂ + -8.88e-16 𝐞₂₄ + -2.22e-16 𝐞₃₄,
 -19.3 + 2e-15 𝐞₁₂ + 3.66e-15 𝐞₁₃ + 3.55e-15 𝐞₁₄ + -1.78e-15 𝐞₂₃ + 3.55e-15 𝐞₂₄ + 3.55e-15 𝐞₃₄ + -8.88e-16 𝐞₁₂₃₄,
 92.9 + -5.33e-15 𝐞₁₃ + -1.42e-14 𝐞₁₄ + -1.24e-14 𝐞₃₄,
 -71.3]

In [66]:
np.roots(char_poly_coefs(fp, alg)), eigvalues

(array([ 4.68691586, -5.08072577, -2.92892743, -1.02165239]),
 array([-5.08072577, -2.92892743, -1.02165239,  4.68691586]))

In [67]:
eigblades = eigenblades_from_values(fp, eigvalues, alg)
[e/norm(e) for e in eigblades], eigvecs

([0.473 𝐞₁ + -0.717 𝐞₂ + 0.347 𝐞₃ + 0.376 𝐞₄,
  -0.451 𝐞₁ + -0.196 𝐞₂ + 0.727 𝐞₃ + -0.478 𝐞₄,
  -0.696 𝐞₁ + -0.0776 𝐞₂ + 0.0163 𝐞₃ + 0.713 𝐞₄,
  -0.296 𝐞₁ + -0.665 𝐞₂ + -0.591 𝐞₃ + -0.348 𝐞₄],
 [0.473 𝐞₁ + -0.717 𝐞₂ + 0.347 𝐞₃ + 0.376 𝐞₄,
  0.451 𝐞₁ + 0.196 𝐞₂ + -0.727 𝐞₃ + 0.478 𝐞₄,
  0.696 𝐞₁ + 0.0776 𝐞₂ + -0.0163 𝐞₃ + -0.713 𝐞₄,
  0.296 𝐞₁ + 0.665 𝐞₂ + 0.591 𝐞₃ + 0.348 𝐞₄])

In [68]:
#4.4 modify mf so to have a repeat eigenvalue (degeneracy)
eigvalues[1] = eigvalues[0]
mf = lambda x: sum(e * P(x, v) for e, v in zip(eigvalues, eigvecs))
for blade in [eigvecs[0], wedge(eigvecs[:2])]:
    of = outermorphism(mf, blade, alg)
    print(terms_ratio(of, blade))

[-5.08072577 -5.08072577 -5.08072577 -5.08072577]
[25.81377431 25.81377431 25.81377431 25.81377431 25.81377431 25.81377431]


In [69]:
eigblades = eigenblades_from_values(mf, set(eigvalues), alg)
[e/norm(e) for e in eigblades], [wedge(eigvecs[:2])] + eigvecs[2:]

([0.416 𝐞₁₂ + -0.501 𝐞₁₃ + 0.0567 𝐞₁₄ + 0.453 𝐞₂₃ + -0.417 𝐞₂₄ + 0.44 𝐞₃₄,
  -0.296 𝐞₁ + -0.665 𝐞₂ + -0.591 𝐞₃ + -0.348 𝐞₄,
  -0.696 𝐞₁ + -0.0776 𝐞₂ + 0.0163 𝐞₃ + 0.713 𝐞₄],
 [0.416 𝐞₁₂ + -0.501 𝐞₁₃ + 0.0567 𝐞₁₄ + 0.453 𝐞₂₃ + -0.417 𝐞₂₄ + 0.44 𝐞₃₄,
  0.696 𝐞₁ + 0.0776 𝐞₂ + -0.0163 𝐞₃ + -0.713 𝐞₄,
  0.296 𝐞₁ + 0.665 𝐞₂ + 0.591 𝐞₃ + 0.348 𝐞₄])

In [70]:
#4.6 a reflection is an orthogonal symmetric transformation 
A3 = create_r_blade(3, alg).normalized()
matrix = trans2matrix(lambda x: (-1)**3*(A3.sw(x)), alg)
# a symmetric matrix, orthogonal, unit eigvals
matrix, [np.dot(matrix[i], matrix[j]) for i,j in [(1,1), (1,2), (2,3), (3,3)]], np.linalg.eigvals(matrix)

(array([[-0.35029372,  0.2162544 , -0.9112839 , -0.00948656],
        [ 0.2162544 , -0.92801983, -0.30332038, -0.0031576 ],
        [-0.9112839 , -0.30332038,  0.27817504,  0.01330594],
        [-0.00948656, -0.0031576 ,  0.01330594, -0.99986148]]),
 [1.0, 1.7740258047294066e-17, 0.0, 0.9999999999999998],
 array([-1.,  1., -1., -1.]))

In [71]:
#4.6 Find blade A from an orthogonal-symmetric-transfromation?
ot = matrix2trans(matrix, alg)
eigvalues, eigvecs = np.linalg.eig(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
A3_ = wedge([v for v, e in zip(eigvecs, eigvalues) if np.isclose(e, -1)])
(-1)**3*(A3.sw(x)), ot(x)

(0.0605 𝐞₁ + -0.778 𝐞₂ + -0.405 𝐞₃ + -0.773 𝐞₄,
 0.0605 𝐞₁ + -0.778 𝐞₂ + -0.405 𝐞₃ + -0.773 𝐞₄)

In [72]:
#4.7 Not true for whole space, but the proper ones
# In matrix terms: diagonal parts commute
eigblades = linalg_eigblades(pf, alg)
eigblades.append(e123)
eigblades, [max_diff(pf(blade.sw(x)), blade.sw(pf(x))) for blade in eigblades]

([-0.512 𝐞₁ + -0.614 𝐞₂ + -0.601 𝐞₃,
  1.0 𝐞₄,
  1.19 𝐞₁₂ + -1.14 𝐞₁₃ + 0.414 𝐞₂₃,
  1 𝐞₁₂₃],
 [0.9095277208554586, 0.0, 6.797924885588128, 0.0])

In [73]:
#4.8 skew transformation -> skew bilinear form -> bivector
fm_biform = lambda vectors: vectors[0] | fm(vectors[1])
F = simplicial_derivative(fm_biform, np.zeros(2), alg)
a | fm(x), a| (x | F)

(0.939, 0.939)

In [74]:
#4.9 so fm(x) is always orthogonal to x
fm(x), x|F, fm(x) | x

(2.13 𝐞₁ + 1.12 𝐞₂ + 0.229 𝐞₃ + -1.79 𝐞₄,
 2.13 𝐞₁ + 1.12 𝐞₂ + 0.229 𝐞₃ + -1.79 𝐞₄,
 )

In [75]:
#4.10 F is a bivector
derivative(fm, x, alg, grade=1), curl(fm, x, alg, grade=1), curl(f, x, alg, grade=1), 2*F

(-1.02e-10 + -4.4 𝐞₁₂ + 1.71 𝐞₁₃ + -0.754 𝐞₁₄ + -1.73 𝐞₂₃ + -3.86 𝐞₂₄ + -2.12 𝐞₃₄,
 -4.4 𝐞₁₂ + 1.71 𝐞₁₃ + -0.754 𝐞₁₄ + -1.73 𝐞₂₃ + -3.86 𝐞₂₄ + -2.12 𝐞₃₄,
 -4.4 𝐞₁₂ + 1.71 𝐞₁₃ + -0.754 𝐞₁₄ + -1.73 𝐞₂₃ + -3.86 𝐞₂₄ + -2.12 𝐞₃₄,
 -4.4 𝐞₁₂ + 1.71 𝐞₁₃ + -0.754 𝐞₁₄ + -1.73 𝐞₂₃ + -3.86 𝐞₂₄ + -2.12 𝐞₃₄)

In [76]:
#4.11 bivector split commutes, you can think about a screw motion splits into translation and rotation
e13*e24, e24*e13

(-1 𝐞₁₂₃₄, -1 𝐞₁₂₃₄)

In [77]:
#4.13 How to factor F?
def wedge_power(A, r):
    if r == 0:
        return 1
    result = A
    for _ in range(r-1):
        result = A ^ result
    return result

Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(3)]
Ck, [char_multi_linear(fm, r, alg) for r in range(1, 5)] # these are full story, but we just take a fraction

([1.0,
  -2.2 𝐞₁₂ + 0.855 𝐞₁₃ + -0.377 𝐞₁₄ + -0.866 𝐞₂₃ + -1.93 𝐞₂₄ + -1.06 𝐞₃₄,
  4.31 𝐞₁₂₃₄],
 [-5.55e-17 + -4.4 𝐞₁₂ + 1.71 𝐞₁₃ + -0.754 𝐞₁₄ + -1.73 𝐞₂₃ + -3.86 𝐞₂₄ + -2.12 𝐞₃₄,
  11.3 + 3.33e-16 𝐞₁₂ + 6.66e-16 𝐞₁₃ + 4.22e-15 𝐞₁₄ + -1.33e-15 𝐞₂₃ + -4.44e-16 𝐞₃₄ + 17.2 𝐞₁₂₃₄,
  -9.12 𝐞₁₂ + 16.6 𝐞₁₃ + -7.46 𝐞₁₄ + -3.25 𝐞₂₃ + -7.37 𝐞₂₄ + -19.0 𝐞₃₄,
  18.6])

In [78]:
[alg.ip(Ck[i-1], Ck[i]) for i in range(1, len(Ck))]

[-2.2 𝐞₁₂ + 0.855 𝐞₁₃ + -0.377 𝐞₁₄ + -0.866 𝐞₂₃ + -1.93 𝐞₂₄ + -1.06 𝐞₃₄,
 4.56 𝐞₁₂ + -8.32 𝐞₁₃ + 3.73 𝐞₁₄ + 1.62 𝐞₂₃ + 3.68 𝐞₂₄ + 9.48 𝐞₃₄]

In [79]:
set.union(*(set(ck.keys()) for ck in Ck if isinstance(ck, MultiVector)))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}

In [80]:
x|F, fm(x), x|F|F, fm(fm(x))

(2.13 𝐞₁ + 1.12 𝐞₂ + 0.229 𝐞₃ + -1.79 𝐞₄,
 2.13 𝐞₁ + 1.12 𝐞₂ + 0.229 𝐞₃ + -1.79 𝐞₄,
 1.6 𝐞₁ + -7.96 𝐞₂ + -1.05 𝐞₃ + -3.22 𝐞₄,
 1.6 𝐞₁ + -7.96 𝐞₂ + -1.05 𝐞₃ + -3.22 𝐞₄)

In [81]:
#4.14 4.18 Why square of eigen values? 
# The odd grade characteristic multivectors have zero scalar
# A correction on Lasenby's paper, not square of f, but the even outermorphism f_(x, y)
Ck2 = [(-1)**(2 - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(3)]
char_poly = char_poly_coefs(fm, alg)
Lk = np.roots(Ck2)

Ck2, char_poly, Lk, np.sqrt(-np.array(Lk)), np.roots(char_poly), char_poly_coefs(lambda x: fm(fm(x)), alg)

([1.0, 11.320285575380566, 18.559067026721404],
 [1, 5.551115123125783e-17, 11.320285575380572, -0.0, 18.559067026721415],
 array([-9.3314027 , -1.98888287]),
 array([3.05473447, 1.41027759]),
 array([-3.52737833e-17+3.05473447j, -3.52737833e-17-3.05473447j,
         7.51820769e-18+1.41027759j,  7.51820769e-18-1.41027759j]),
 [1,
  22.640571150761136,
  165.26699956161224,
  420.18787751023126,
  344.4389689023381])

In [82]:
Ck[1], Ck[1] | Ck[2]

(-2.2 𝐞₁₂ + 0.855 𝐞₁₃ + -0.377 𝐞₁₄ + -0.866 𝐞₂₃ + -1.93 𝐞₂₄ + -1.06 𝐞₃₄,
 4.56 𝐞₁₂ + -8.32 𝐞₁₃ + 3.73 𝐞₁₄ + 1.62 𝐞₂₃ + 3.68 𝐞₂₄ + 9.48 𝐞₃₄)

In [83]:
#4.15
mv_map1 = {}
mv_map2 = {}
ck1, ck2 = Ck[1], Ck[1] | Ck[2]
for key in ck2.keys():
    if np.isclose(ck1[key],0) and np.isclose(ck2[key],0):
        continue
    ans = np.linalg.solve(np.array([[1, 1], Lk]), [ck1[key], ck2[key]])
    mv_map1[key], mv_map2[key] = ans
F1 = alg.multivector(mv_map1)
F2 = alg.multivector(mv_map2)
F1 + F2, F, F1*F2, F2*F1, F**2, F1**2, F2**2


(-2.2 𝐞₁₂ + 0.855 𝐞₁₃ + -0.377 𝐞₁₄ + -0.866 𝐞₂₃ + -1.93 𝐞₂₄ + -1.06 𝐞₃₄,
 -2.2 𝐞₁₂ + 0.855 𝐞₁₃ + -0.377 𝐞₁₄ + -0.866 𝐞₂₃ + -1.93 𝐞₂₄ + -1.06 𝐞₃₄,
 -1.24e-15 + -1.12e-16 𝐞₁₂ + 6.59e-17 𝐞₁₃ + 1.11e-16 𝐞₁₄ + -1.11e-16 𝐞₂₄ + 2.08e-17 𝐞₃₄ + 4.31 𝐞₁₂₃₄,
 -1.24e-15 + 1.11e-16 𝐞₁₂ + -6.59e-17 𝐞₁₃ + -1.11e-16 𝐞₁₄ + -1.03e-16 𝐞₂₃ + 6.6e-17 𝐞₂₄ + -2.08e-17 𝐞₃₄ + 4.31 𝐞₁₂₃₄,
 -11.3 + -5.55e-17 𝐞₁₃ + 8.62 𝐞₁₂₃₄,
 -1.99 + 7.81e-16 𝐞₁₂₃₄,
 -9.33 + -1.03e-16 𝐞₁₃ + 3.82e-16 𝐞₁₄ + 1.71e-15 𝐞₁₂₃₄)

In [84]:
#4.16
def normsq(A):
    return A.reverse().sp(A)[0]
(
    [(-normsq(F) - np.sqrt((normsq(F)**2 - normsq(F ^ F))))/2,
    (-normsq(F) + np.sqrt((normsq(F)**2 - normsq(F ^ F))))/2],
    Lk
)


([-9.331402700897318, -1.9888828744832479], array([-9.3314027 , -1.98888287]))

In [85]:
#4.17
F2_ = F / (1 + (1/2/Lk[0])*(F ^ F))
F1_ = F / (1 + (1/2/Lk[1])*(F ^ F))
F1_, F1, F2_, F2

(-0.0247 𝐞₁₂ + 0.902 𝐞₁₃ + -0.406 𝐞₁₄ + 0.0134 𝐞₂₃ + 0.0215 𝐞₂₄ + -1.0 𝐞₃₄,
 -0.0247 𝐞₁₂ + 0.902 𝐞₁₃ + -0.406 𝐞₁₄ + 0.0134 𝐞₂₃ + 0.0215 𝐞₂₄ + -1.0 𝐞₃₄,
 -2.18 𝐞₁₂ + -0.0465 𝐞₁₃ + 0.0291 𝐞₁₄ + -0.88 𝐞₂₃ + -1.95 𝐞₂₄ + -0.0535 𝐞₃₄,
 -2.18 𝐞₁₂ + -0.0465 𝐞₁₃ + 0.0291 𝐞₁₄ + -0.88 𝐞₂₃ + -1.95 𝐞₂₄ + -0.0535 𝐞₃₄)

In [86]:
def bivector_split(F, alg: Algebra):
    m = alg.d // 2
    if m <= 1:
        return F
    Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(m+1)]
    Ck2 = [(-1)**(m - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(m+1)]
    Lk = np.roots(Ck2)
    mv_map_list = [{} for _ in range(m)]
    ck_inner_list = [alg.ip(Ck[i-1], Ck[i]) for i in range(1, m+1)]
    mv_keys = set.union(*(set(ck.keys()) for ck in ck_inner_list if isinstance(ck, MultiVector)))
    linear_eq_matrix = np.array([[np.prod(Lk[i:i+k]) for i in range(m)] for k in range(m)])
    inv_matrix = np.linalg.inv(linear_eq_matrix)
    for key in mv_keys:
        if all([np.isclose(ck[key],0) for ck in ck_inner_list]):
            continue
        ans = np.dot(inv_matrix, [ck[key] for ck in ck_inner_list])
        for i, v in enumerate(ans):
            mv_map_list[i][key] = v
    return [alg.multivector(mv_map) for mv_map in mv_map_list]

In [87]:
# From Graded Symmetry Groups: Plane and Simple, 
# instead of solving linear equations, the solutions are given algebraically
def bivector_split_(F, alg: Algebra):  
    # FIXME: it produce commuting bivectors but not blades
    m = alg.d // 2
    if m <= 1:
        return F
    Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(m+1)]
    Ck2 = [(-1)**(m - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(m+1)]
    Lk = np.roots(Ck2)
    blades = []
    for l in Lk:
        even = sum(l ** r * c for r, c in enumerate(Ck[-1::-2]))
        odd = sum(l ** r * c for r, c in enumerate(Ck[-2::-2]))
        blades.append(even * odd.reverse() / normsq(odd)) # FIXME: the inverse is wrong
    return blades

In [88]:
# What if we use the above function to split a bivector that's almost a blade?
# The result blades are R itself and its dual with a small magnitude
# Should assert not simple before process?

R = create_r_blade(2, alg) + 1e-12 * A # a blade with a small perturbation
blades = bivector_split(R, alg)
R**2, [terms_ratio(b, r) for r in [R, R.dual()] for b in blades]

(-1.27 + 1.23e-12 𝐞₁ + 1.6e-12 𝐞₂ + -8.15e-13 𝐞₃ + -1.03e-12 𝐞₄ + 7.41e-13 𝐞₁₂ + -3.77e-13 𝐞₁₃ + 2.68e-13 𝐞₁₄ + -4.13e-13 𝐞₂₃ + 5.23e-13 𝐞₂₄ + -7.51e-13 𝐞₃₄ + -2.27e-14 𝐞₁₂₃ + -4.36e-13 𝐞₁₂₄ + -4.98e-13 𝐞₁₃₄ + 9.86e-14 𝐞₂₃₄ + -1.43e-12 𝐞₁₂₃₄,
 [array([ 5.50531314e-13, -4.71499383e-13,  2.49815652e-13,  1.27527697e-12,
         -6.75683066e-13,  5.78684882e-13]),
  array([1., 1., 1., 1., 1., 1.]),
  array([5.6443259e-13, 5.6443259e-13, 5.6443259e-13, 5.6443259e-13,
         5.6443259e-13, 5.6443259e-13]),
  array([ 1.02525065, -1.19710144,  2.25939641,  0.44259608, -0.8353511 ,
          0.97537124])])

In [89]:
# double skew is symmetric
fm2 = lambda x: fm(fm(x))
curl(fm2, x, alg, h=1e-2, grade=1)

1.28e-13 𝐞₁₂ + -7.11e-15 𝐞₁₃ + -4.26e-14 𝐞₁₄ + -1.85e-13 𝐞₂₃ + 5.68e-14 𝐞₂₄ + 4.97e-14 𝐞₃₄

In [90]:
#4.19
v = P(e1, F1)
fm2(v), (F1 ** 2) * v

(-0.979 𝐞₁ + -0.00341 𝐞₂ + -0.408 𝐞₃ + -0.907 𝐞₄,
 -0.979 𝐞₁ + -0.00341 𝐞₂ + -0.408 𝐞₃ + -0.907 𝐞₄ + 3.56e-16 𝐞₁₂₃ + -1.6e-16 𝐞₁₂₄ + 1.34e-18 𝐞₁₃₄ + -3.84e-16 𝐞₂₃₄)

In [91]:
#4.20 
F1_eigvecs= [v.normalized() for v in gram_schmidt(blade_split(F1, alg))]
F1, np.sqrt(normsq(F1)) * wedge(F1_eigvecs)


(-0.0247 𝐞₁₂ + 0.902 𝐞₁₃ + -0.406 𝐞₁₄ + 0.0134 𝐞₂₃ + 0.0215 𝐞₂₄ + -1.0 𝐞₃₄,
 -0.0247 𝐞₁₂ + 0.902 𝐞₁₃ + -0.406 𝐞₁₄ + 0.0134 𝐞₂₃ + 0.0215 𝐞₂₄ + -1.0 𝐞₃₄)

In [92]:
# If Lk[0] == Lk[1], the linear equation has no unique solution in 4.15
# nonunique bivector split? We should find another split of e12 + e34. 
t = 1.1
R = blade_exp(t*e13) * blade_exp(t*e24)
a,b,c,d = [R.sw(v) for v in alg.frame]
a,b,c,d, a*b+c*d, a*b, c*d

(-0.589 𝐞₁ + -0.808 𝐞₃,
 -0.589 𝐞₂ + -0.808 𝐞₄,
 0.808 𝐞₁ + -0.589 𝐞₃,
 0.808 𝐞₂ + -0.589 𝐞₄,
 1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 0.346 𝐞₁₂ + 0.476 𝐞₁₄ + -0.476 𝐞₂₃ + 0.654 𝐞₃₄,
 0.654 𝐞₁₂ + -0.476 𝐞₁₄ + 0.476 𝐞₂₃ + 0.346 𝐞₃₄)

In [93]:
R.sw(e12 + e34), (e12+e34) * (R**2), R**2

(1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 0.346 + -0.476 𝐞₁₃ + -0.476 𝐞₂₄ + -0.654 𝐞₁₂₃₄)

In [94]:
#4.21
B2 = B.grade(2)
# fm = lambda x: f(x) - adjoint_outermorphism(f, x, alg)
# F = curl(fm, x, alg, grade=1) / 2
outermorphism(fm, B2, alg), (0.5 * (B2 | (F^F))) - ((B2|F) * F)

(2.87 𝐞₁₂ + -0.0196 𝐞₁₃ + -2.39 𝐞₁₄ + -2.13 𝐞₂₃ + 7.09 𝐞₂₄ + 1.35 𝐞₃₄,
 2.87 𝐞₁₂ + -0.0196 𝐞₁₃ + -2.39 𝐞₁₄ + -2.13 𝐞₂₃ + 7.09 𝐞₂₄ + 1.35 𝐞₃₄)

In [95]:
# What is a symmetric bivector function?
# I guess it means:
outermorphism(fm, B2, alg), adjoint_outermorphism(fm, B2, alg)

(2.87 𝐞₁₂ + -0.0196 𝐞₁₃ + -2.39 𝐞₁₄ + -2.13 𝐞₂₃ + 7.09 𝐞₂₄ + 1.35 𝐞₃₄,
 2.87 𝐞₁₂ + -0.0196 𝐞₁₃ + -2.39 𝐞₁₄ + -2.13 𝐞₂₃ + 7.09 𝐞₂₄ + 1.35 𝐞₃₄)

In [96]:
#4.23 The bivector split solves the eigenbivector problem of a skew transformation
outermorphism(fm, F1, alg), normsq(F1)*F1

(-0.0491 𝐞₁₂ + 1.79 𝐞₁₃ + -0.808 𝐞₁₄ + 0.0267 𝐞₂₃ + 0.0427 𝐞₂₄ + -2.0 𝐞₃₄,
 -0.0491 𝐞₁₂ + 1.79 𝐞₁₃ + -0.808 𝐞₁₄ + 0.0267 𝐞₂₃ + 0.0427 𝐞₂₄ + -2.0 𝐞₃₄)

In [97]:
#5.1 normal transformation
af = lambda x: adjoint_outermorphism(f, x, alg)

fp(fm(x)), fm(fp(x)), af(f(x)), f(af(x)) # not normal

(-0.0606 𝐞₁ + 1.03 𝐞₂ + 3.55 𝐞₃ + 4.05 𝐞₄,
 0.688 𝐞₁ + 2.43 𝐞₂ + 0.472 𝐞₃ + -8.38 𝐞₄,
 3.0 𝐞₁ + 25.1 𝐞₂ + 10.6 𝐞₃ + 23.3 𝐞₄,
 4.5 𝐞₁ + 27.9 𝐞₂ + 4.47 𝐞₃ + -1.61 𝐞₄)

In [98]:
M = blade_exp(create_r_blade(2, alg))
Mdual = blade_split(M.grade(2).dual(), alg)
rigid = lambda x: 1.1 *  M.sw(x) - 4.4 * P(x, Mdual[0]) + 3.3 * P(x, Mdual[1])
adrigid = lambda x: adjoint_outermorphism(rigid, x, alg)
adrigid(rigid(x)), rigid(adrigid(x))

(-4.48 𝐞₁ + 11.8 𝐞₂ + -4.12 𝐞₃ + 4.11 𝐞₄,
 -4.48 𝐞₁ + 11.8 𝐞₂ + -4.12 𝐞₃ + 4.11 𝐞₄)

In [99]:
rigid(x), 3.3 * M**2 * P(x, M.grade(2).normalized()) + 3.3 * P(x, M.grade(2).normalized().dual()) - 4.4 * P(x, Mdual[0]) + 5.5 * P(x, Mdual[1])

(-1.29 𝐞₁ + 2.49 𝐞₂ + 0.635 𝐞₃ + -1.98 𝐞₄,
 -2.08 𝐞₁ + 5.29 𝐞₂ + -1.55 𝐞₃ + -2.46 𝐞₄ + -1.48e-16 𝐞₁₂₃ + 3.93e-16 𝐞₁₂₄ + 2.17e-16 𝐞₁₃₄ + 1.37e-16 𝐞₂₃₄)

In [100]:
#5.2
def norm(A):
    return np.sqrt(normsq(A))
b1, b2 = norm(F1), norm(F2)
i1, i2 = F1/b1, F2/b2

F, b1*i1 + b2*i2, outermorphism(fm, i1, alg), b1**2*i1

(-2.2 𝐞₁₂ + 0.855 𝐞₁₃ + -0.377 𝐞₁₄ + -0.866 𝐞₂₃ + -1.93 𝐞₂₄ + -1.06 𝐞₃₄,
 -2.2 𝐞₁₂ + 0.855 𝐞₁₃ + -0.377 𝐞₁₄ + -0.866 𝐞₂₃ + -1.93 𝐞₂₄ + -1.06 𝐞₃₄,
 -0.0348 𝐞₁₂ + 1.27 𝐞₁₃ + -0.573 𝐞₁₄ + 0.019 𝐞₂₃ + 0.0303 𝐞₂₄ + -1.42 𝐞₃₄,
 -0.0348 𝐞₁₂ + 1.27 𝐞₁₃ + -0.573 𝐞₁₄ + 0.019 𝐞₂₃ + 0.0303 𝐞₂₄ + -1.42 𝐞₃₄)

In [101]:
# 5.3
terms_ratio(outermorphism(fp, i1, alg), i1) # not normal

array([ 4.22436235e+02,  8.25189238e-02,  4.74747073e+00, -8.10224999e+02,
        5.88651531e+02, -1.91514808e+00])

In [102]:
# 5.3
def sym_part(f, alg):
    return lambda x: (f(x) + adjoint_outermorphism(f, x, alg))/2

def skew_part(f, alg):
    return lambda x: (f(x) - adjoint_outermorphism(f, x, alg))/2

R = curl(rigid, x, alg, grade=1)/2
skew_blades = bivector_split(R, alg)
skew_eigs = [norm(b) for b in skew_blades]
skew_blades = [b / e for b,e in zip(skew_blades, skew_eigs) if e > 1e-6]
skew_eigs = [e for e in skew_eigs if e > 1e-6]
rigid_p = sym_part(rigid, alg)
[terms_ratio(outermorphism(rigid_p, i, alg), i) for i in skew_blades]

[array([0.47922398, 0.47922397, 0.47922397, 0.47922397, 0.47922397,
        0.47922397])]

In [103]:
#5.4
rand_vec = create_r_vectors(1, alg)[0]
sym_vecs = [(rand_vec | i).normalized() for i in skew_blades]
other_sym_space = wedge(skew_blades).dual()
sym_eigs = [terms_ratio(rigid_p(a), a)[0] for a in sym_vecs]

sym_eigs, np.square(sym_eigs), [terms_ratio(outermorphism(rigid_p, i, alg), i)[0] for i in skew_blades]


([-0.6922600476481583], array([0.47922397]), [0.47922398248691594])

In [104]:
#5.5a
m = len(skew_blades)
rigid_eigs = [sym_eigs[k] + skew_blades[k] * skew_eigs[k] for k in range(m)]
[rigid(a) for a in sym_vecs], [a*l for a, l in zip(sym_vecs,rigid_eigs)], [l.reverse()*a for a, l in zip(sym_vecs,rigid_eigs)]

([-0.959 𝐞₁ + -0.494 𝐞₂ + -0.211 𝐞₃ + -0.0293 𝐞₄],
 [-0.959 𝐞₁ + -0.494 𝐞₂ + -0.211 𝐞₃ + -0.0293 𝐞₄ + -6.94e-18 𝐞₁₂₃ + -2.78e-17 𝐞₁₂₄ + -2.78e-17 𝐞₂₃₄],
 [-0.959 𝐞₁ + -0.494 𝐞₂ + -0.211 𝐞₃ + -0.0293 𝐞₄ + 2.78e-17 𝐞₁₂₄ + -1.39e-17 𝐞₁₃₄ + 2.08e-17 𝐞₂₃₄])

In [105]:
#5.5b
[adjoint_outermorphism(rigid, a, alg) for a in sym_vecs], [l*a for a, l in zip(sym_vecs,rigid_eigs)], [a*l.reverse() for a, l in zip(sym_vecs,rigid_eigs)]

([0.0307 𝐞₁ + 0.0769 𝐞₂ + 0.762 𝐞₃ + 0.789 𝐞₄],
 [0.0307 𝐞₁ + 0.0769 𝐞₂ + 0.762 𝐞₃ + 0.789 𝐞₄ + -2.78e-17 𝐞₁₂₄ + 1.39e-17 𝐞₁₃₄ + -2.08e-17 𝐞₂₃₄],
 [0.0307 𝐞₁ + 0.0769 𝐞₂ + 0.762 𝐞₃ + 0.789 𝐞₄ + 6.94e-18 𝐞₁₂₃ + 2.78e-17 𝐞₁₂₄ + 2.78e-17 𝐞₂₃₄])

In [106]:
[norm(l) for l in rigid_eigs], [np.sqrt(sym_eigs[k] ** 2 + skew_eigs[k] ** 2) for k in range(m)]

([1.1000000005681279], [1.1000000005681279])

In [107]:
#5.6
rho = [norm(l) for l in rigid_eigs] # dilation
theta = [np.arccos(sym_eigs[k] / rho[k]) for k in range(m)] # rotation angle

rigid_eigs, [rho[k] * blade_exp(theta[k] * skew_blades[k]) for k in range(m)] # skew_blades gives rotation plane

([-0.692 + -0.0423 𝐞₁₂ + -0.523 𝐞₁₃ + -0.546 𝐞₁₄ + -0.26 𝐞₂₃ + -0.28 𝐞₂₄ + -0.104 𝐞₃₄],
 [-0.692 + -0.0423 𝐞₁₂ + -0.523 𝐞₁₃ + -0.546 𝐞₁₄ + -0.26 𝐞₂₃ + -0.28 𝐞₂₄ + -0.104 𝐞₃₄])

In [108]:
np.sqrt(1 - (skew_eigs[0]/rho[0])**2) * rho[0]

0.6922600476481585

In [109]:
# spectral decomposition of symmetric part in other_sym_space
other_sym_frame = blade_split(other_sym_space, alg)
other_sym_eigs = np.roots(char_poly_coefs(rigid_p, alg, other_sym_frame, reciprocal(other_sym_frame)))
other_sym_eigs

array([ 4.37738544, -3.27738544])

In [110]:
#5.9 spectral decomposition of a normal transformation
sym_vecs_dual = [a|i for a, i in zip(sym_vecs, skew_blades)]
other_sym_blades = eigenblades_from_values(rigid_p, other_sym_eigs, alg, other_sym_frame)

rigid(x), sum([P(x, a) * l for a, l in zip(sym_vecs + sym_vecs_dual + other_sym_blades, rigid_eigs * 2 + list(other_sym_eigs))])

(-1.29 𝐞₁ + 2.49 𝐞₂ + 0.635 𝐞₃ + -1.98 𝐞₄,
 -1.29 𝐞₁ + 2.49 𝐞₂ + 0.635 𝐞₃ + -1.98 𝐞₄ + 1.67e-17 𝐞₁₂₃ + 2.13e-17 𝐞₁₂₄ + 2.78e-17 𝐞₁₃₄ + 2.95e-17 𝐞₂₃₄)

In [111]:
#5.17
sym_eigs, sym_vecs = np.linalg.eigh(trans2matrix(rigid_p, alg))
sym_vecs = [alg.vector(v) for v in sym_vecs.transpose()]
sym_eigs, sym_vecs

(array([-3.27738544, -0.69226005, -0.69226005,  4.37738544]),
 [0.109 𝐞₁ + 0.0559 𝐞₂ + -0.719 𝐞₃ + 0.684 𝐞₄,
  0.886 𝐞₁ + 0.446 𝐞₂ + 0.0709 𝐞₃ + -0.103 𝐞₄,
  0.000223 𝐞₁ + 0.056 𝐞₂ + 0.691 𝐞₃ + 0.721 𝐞₄,
  0.451 𝐞₁ + -0.891 𝐞₂ + 0.0338 𝐞₃ + 0.0367 𝐞₄])

In [112]:
# In my case, all vectors are both eigenvectors of rigid_p and rigid_n
rigid_n = lambda x: rigid(adjoint_outermorphism(rigid, x, alg))
[terms_ratio(rigid_n(v), v) for v in sym_vecs]

[array([10.74125531, 10.74125531, 10.74125531, 10.74125531]),
 array([1.21, 1.21, 1.21, 1.21]),
 array([1.21, 1.21, 1.21, 1.21]),
 array([19.16150327, 19.16150327, 19.16150327, 19.16150327])]

In [113]:
# the same spectral decomposition of the normal transformation
beta = [v ^ rigid(v) for v in sym_vecs]
rigid_eigs = [a + b for a,b in zip(sym_eigs, beta)]
rigid(x), sum([P(x, a) * l for a, l in zip(sym_vecs, rigid_eigs)])

(-1.29 𝐞₁ + 2.49 𝐞₂ + 0.635 𝐞₃ + -1.98 𝐞₄,
 -1.29 𝐞₁ + 2.49 𝐞₂ + 0.635 𝐞₃ + -1.98 𝐞₄ + 7.43e-18 𝐞₁₂₃ + 9.38e-18 𝐞₁₂₄ + 3.8e-17 𝐞₁₃₄ + 5.44e-18 𝐞₂₃₄)

In [114]:
#5.19 our function "rigid" is normal but not orthogonal
A, outermorphism(rigid, adjoint_outermorphism(rigid, A, alg, h=1e-2), alg, h=1e-2)

(0.163 + 0.88 𝐞₁ + 0.357 𝐞₂ + 0.64 𝐞₃ + 0.663 𝐞₄ + 0.974 𝐞₁₂ + 0.135 𝐞₁₃ + 0.247 𝐞₁₄ + 0.502 𝐞₂₃ + 0.971 𝐞₂₄ + 0.325 𝐞₃₄ + 0.352 𝐞₁₂₃ + 0.728 𝐞₁₂₄ + 0.392 𝐞₁₃₄ + 0.697 𝐞₂₃₄ + 0.568 𝐞₁₂₃₄,
 0.163 + 2.19 𝐞₁ + -1.51 𝐞₂ + 0.103 𝐞₃ + 1.59 𝐞₄ + 19.3 𝐞₁₂ + 0.434 𝐞₁₃ + -12.4 𝐞₁₄ + -5.36 𝐞₂₃ + 28.2 𝐞₂₄ + 2.02 𝐞₃₄ + -14.0 𝐞₁₂₃ + 63.1 𝐞₁₂₄ + -51.5 𝐞₁₃₄ + 1.2e+02 𝐞₂₃₄ + 1.71e+02 𝐞₁₂₃₄)

In [115]:
#5.19 But if we normalize each "eigenvalues" in the spectral decomposition
ortho_eigs = [l.normalized() for l in rigid_eigs]
ortho = lambda x: sum([P(x, a) * l for a, l in zip(sym_vecs, ortho_eigs)])
A, outermorphism(ortho, adjoint_outermorphism(ortho, A, alg, h=1e-2), alg, h=1e-2)

(0.163 + 0.88 𝐞₁ + 0.357 𝐞₂ + 0.64 𝐞₃ + 0.663 𝐞₄ + 0.974 𝐞₁₂ + 0.135 𝐞₁₃ + 0.247 𝐞₁₄ + 0.502 𝐞₂₃ + 0.971 𝐞₂₄ + 0.325 𝐞₃₄ + 0.352 𝐞₁₂₃ + 0.728 𝐞₁₂₄ + 0.392 𝐞₁₃₄ + 0.697 𝐞₂₃₄ + 0.568 𝐞₁₂₃₄,
 0.163 + 0.88 𝐞₁ + 0.357 𝐞₂ + 0.64 𝐞₃ + 0.663 𝐞₄ + 0.974 𝐞₁₂ + 0.135 𝐞₁₃ + 0.247 𝐞₁₄ + 0.502 𝐞₂₃ + 0.971 𝐞₂₄ + 0.325 𝐞₃₄ + 0.352 𝐞₁₂₃ + 0.728 𝐞₁₂₄ + 0.392 𝐞₁₃₄ + 0.697 𝐞₂₃₄ + 0.568 𝐞₁₂₃₄)

In [116]:
#5.20 left/right product of exponential
ortho(x), sum([l.inv() * P(x, a) for a, l in zip(sym_vecs, ortho_eigs)])

(-0.176 𝐞₁ + 0.597 𝐞₂ + -0.144 𝐞₃ + -0.981 𝐞₄ + 8.78e-18 𝐞₁₂₃ + -6.73e-19 𝐞₁₂₄ + 7.95e-18 𝐞₁₃₄ + 1.43e-17 𝐞₂₃₄,
 -0.176 𝐞₁ + 0.597 𝐞₂ + -0.144 𝐞₃ + -0.981 𝐞₄ + 1.55e-17 𝐞₁₂₃ + -1.5e-17 𝐞₁₂₄ + -6.9e-18 𝐞₁₃₄ + -3.33e-17 𝐞₂₃₄)

In [117]:
#5.21
(
    [sym_vecs[k-1].normalized() * sym_vecs[k].normalized() for k in range(1, 4)],
    [e.grade(2).normalized() for e in ortho_eigs]
)


([9.71e-17 + -0.00101 𝐞₁₂ + 0.645 𝐞₁₃ + -0.617 𝐞₁₄ + 0.325 𝐞₂₃ + -0.311 𝐞₂₄ + 0.0254 𝐞₃₄,
  -2.78e-17 + 0.0495 𝐞₁₂ + 0.612 𝐞₁₃ + 0.639 𝐞₁₄ + 0.304 𝐞₂₃ + 0.328 𝐞₂₄ + 0.122 𝐞₃₄,
  2.08e-17 + -0.0254 𝐞₁₂ + -0.311 𝐞₁₃ + -0.325 𝐞₁₄ + 0.617 𝐞₂₃ + 0.645 𝐞₂₄ + 0.00101 𝐞₃₄],
 [0.0757 𝐞₁₂ + 0.238 𝐞₁₃ + -0.125 𝐞₁₄ + 0.524 𝐞₂₃ + -0.481 𝐞₂₄ + -0.645 𝐞₃₄,
  -0.0495 𝐞₁₂ + -0.612 𝐞₁₃ + -0.639 𝐞₁₄ + -0.304 𝐞₂₃ + -0.328 𝐞₂₄ + -0.122 𝐞₃₄,
  -0.0495 𝐞₁₂ + -0.612 𝐞₁₃ + -0.639 𝐞₁₄ + -0.304 𝐞₂₃ + -0.328 𝐞₂₄ + -0.122 𝐞₃₄,
  -0.316 𝐞₁₃ + 0.316 𝐞₁₄ + 0.632 𝐞₂₃ + -0.632 𝐞₂₄ + 0.0488 𝐞₃₄])

In [118]:
#5.23
def simple_rotor_sqrt(R):
    # for rotations only, don't use on screw motions
    R_norm = norm(R+1)
    assert R_norm >= 1e-4, "no explicit square root for -1"
    return (R + 1)/R_norm
    
neg_eig_pairs = []
eig_rotors = []
for i, e in enumerate(ortho_eigs):
    if normsq(e + 1) <= 1e-4:
        neg_eig_pairs.append((-1, sym_vecs[i]))
    else:
        eig_rotors.append(simple_rotor_sqrt(e))

sym_vecs, [[e.reverse().sw(a) for e in eig_rotors] for a in sym_vecs]

([0.109 𝐞₁ + 0.0559 𝐞₂ + -0.719 𝐞₃ + 0.684 𝐞₄,
  0.886 𝐞₁ + 0.446 𝐞₂ + 0.0709 𝐞₃ + -0.103 𝐞₄,
  0.000223 𝐞₁ + 0.056 𝐞₂ + 0.691 𝐞₃ + 0.721 𝐞₄,
  0.451 𝐞₁ + -0.891 𝐞₂ + 0.0338 𝐞₃ + 0.0367 𝐞₄],
 [[0.109 𝐞₁ + 0.0559 𝐞₂ + -0.719 𝐞₃ + 0.684 𝐞₄,
   0.109 𝐞₁ + 0.0559 𝐞₂ + -0.719 𝐞₃ + 0.684 𝐞₄,
   0.109 𝐞₁ + 0.0559 𝐞₂ + -0.719 𝐞₃ + 0.684 𝐞₄],
  [-0.558 𝐞₁ + -0.324 𝐞₂ + -0.581 𝐞₃ + -0.496 𝐞₄,
   -0.558 𝐞₁ + -0.324 𝐞₂ + -0.581 𝐞₃ + -0.496 𝐞₄,
   0.886 𝐞₁ + 0.446 𝐞₂ + 0.0709 𝐞₃ + -0.103 𝐞₄],
  [0.688 𝐞₁ + 0.312 𝐞₂ + -0.379 𝐞₃ + -0.534 𝐞₄,
   0.688 𝐞₁ + 0.312 𝐞₂ + -0.379 𝐞₃ + -0.534 𝐞₄,
   0.000223 𝐞₁ + 0.056 𝐞₂ + 0.691 𝐞₃ + 0.721 𝐞₄],
  [0.451 𝐞₁ + -0.891 𝐞₂ + 0.0338 𝐞₃ + 0.0367 𝐞₄,
   0.451 𝐞₁ + -0.891 𝐞₂ + 0.0338 𝐞₃ + 0.0367 𝐞₄,
   0.451 𝐞₁ + -0.891 𝐞₂ + 0.0338 𝐞₃ + 0.0367 𝐞₄]])

In [119]:
#5.25 I selected an example with -1 as an eigenvalue, 
# so we need an extra fix for the canonical form.
theta = reduce(alg.gp, eig_rotors[::2])
(
    ortho(x),
    theta.reverse().sw(x) - (2*sum(P(x, neg_eig_pairs[i][1]) for i in range(len(neg_eig_pairs))))
)


(-0.176 𝐞₁ + 0.597 𝐞₂ + -0.144 𝐞₃ + -0.981 𝐞₄ + 8.78e-18 𝐞₁₂₃ + -6.73e-19 𝐞₁₂₄ + 7.95e-18 𝐞₁₃₄ + 1.43e-17 𝐞₂₃₄,
 -0.176 𝐞₁ + 0.597 𝐞₂ + -0.144 𝐞₃ + -0.981 𝐞₄)

In [120]:
#5.28
blade_exp(F1) * blade_exp(F2), blade_exp(F2) * blade_exp(F1), 

(-0.159 + 0.00733 𝐞₁₂ + -0.629 𝐞₁₃ + 0.283 𝐞₁₄ + -0.0134 𝐞₂₃ + -0.0238 𝐞₂₄ + 0.7 𝐞₃₄ + 0.0856 𝐞₁₂₃₄,
 -0.159 + 0.00733 𝐞₁₂ + -0.629 𝐞₁₃ + 0.283 𝐞₁₄ + -0.0134 𝐞₂₃ + -0.0238 𝐞₂₄ + 0.7 𝐞₃₄ + 0.0856 𝐞₁₂₃₄)

In [121]:
#5.29
blade_exp_ = blade_exp(F1)
u1 = P(e1, F1).normalized()
u2 = u1 | blade_exp_
blade_exp_, u1 * u2

(0.16 + -0.0173 𝐞₁₂ + 0.631 𝐞₁₃ + -0.284 𝐞₁₄ + 0.00941 𝐞₂₃ + 0.015 𝐞₂₄ + -0.703 𝐞₃₄,
 0.16 + -0.0173 𝐞₁₂ + 0.631 𝐞₁₃ + -0.284 𝐞₁₄ + 0.00941 𝐞₂₃ + 0.015 𝐞₂₄ + -0.703 𝐞₃₄)

In [122]:
# reflection as sandwich product
x - 2 * P(x, neg_eig_pairs[0][1]), -neg_eig_pairs[0][1].sw(x)

(0.0666 𝐞₁ + 0.798 𝐞₂ + 0.853 𝐞₃ + 0.0125 𝐞₄,
 0.0666 𝐞₁ + 0.798 𝐞₂ + 0.853 𝐞₃ + 0.0125 𝐞₄)

In [123]:
#5.30
theta2 = theta * neg_eig_pairs[0][1]
ortho(x), -neg_eig_pairs[0][1].sw(theta.reverse().sw(x)), -theta2.reverse().sw(x)

(-0.176 𝐞₁ + 0.597 𝐞₂ + -0.144 𝐞₃ + -0.981 𝐞₄ + 8.78e-18 𝐞₁₂₃ + -6.73e-19 𝐞₁₂₄ + 7.95e-18 𝐞₁₃₄ + 1.43e-17 𝐞₂₃₄,
 -0.176 𝐞₁ + 0.597 𝐞₂ + -0.144 𝐞₃ + -0.981 𝐞₄,
 -0.176 𝐞₁ + 0.597 𝐞₂ + -0.144 𝐞₃ + -0.981 𝐞₄)

In [124]:
#5.37a
outermorphism(ortho, A3, alg), (-1)**(3*1)*theta2.reverse().sw(A3)

(-0.409 𝐞₁₂₃ + -0.361 𝐞₁₂₄ + -0.434 𝐞₁₃₄ + 0.717 𝐞₂₃₄,
 -0.409 𝐞₁₂₃ + -0.361 𝐞₁₂₄ + -0.434 𝐞₁₃₄ + 0.717 𝐞₂₃₄)

In [125]:
#5.37b
adjoint_outermorphism(ortho, F2, alg), (-1)**(2*1)*theta2.sw(F2)

(-0.119 𝐞₁₂ + -1.31 𝐞₁₃ + -1.81 𝐞₁₄ + 1.21 𝐞₂₃ + 1.69 𝐞₂₄ + 0.0943 𝐞₃₄,
 -0.119 𝐞₁₂ + -1.31 𝐞₁₃ + -1.81 𝐞₁₄ + 1.21 𝐞₂₃ + 1.69 𝐞₂₄ + 0.0943 𝐞₃₄)

In [126]:
#5.38
outermorphism(ortho, A3 * F2, alg), outermorphism(ortho, A3, alg) * outermorphism(ortho, F2, alg)

(-0.732 𝐞₁ + 0.636 𝐞₂ + 1.21 𝐞₃ + 1.68 𝐞₄ + 0.676 𝐞₁₂₃ + -0.444 𝐞₁₂₄ + -1.66 𝐞₁₃₄ + -0.841 𝐞₂₃₄,
 -0.732 𝐞₁ + 0.636 𝐞₂ + 1.21 𝐞₃ + 1.68 𝐞₄ + 0.676 𝐞₁₂₃ + -0.444 𝐞₁₂₄ + -1.66 𝐞₁₃₄ + -0.841 𝐞₂₃₄)

In [144]:
#5.39
from scipy.stats import ortho_group
matrix = ortho_group.rvs(dim=5)
alg = Algebra(5)
ortho = matrix2trans(matrix, alg)
x = create_r_vectors(1, alg)[0]
ortho_curl = curl(ortho, x, alg, grade=1)
ortho_curl ** 2

-2.76 + 1.39e-17 𝐞₁₂ + 6.94e-18 𝐞₁₃ + 5.55e-17 𝐞₁₅ + -2.08e-17 𝐞₂₅ + 0.735 𝐞₁₂₃₄ + -0.649 𝐞₁₂₃₅ + 0.526 𝐞₁₂₄₅ + 0.586 𝐞₁₃₄₅ + -1.17 𝐞₂₃₄₅

In [145]:
# check orthogonality
x, ortho(adjoint_outermorphism(ortho, x, alg, h=1e-2))

(0.144 𝐞₁ + 0.962 𝐞₂ + 0.543 𝐞₃ + 0.724 𝐞₄ + 0.346 𝐞₅,
 0.144 𝐞₁ + 0.962 𝐞₂ + 0.543 𝐞₃ + 0.724 𝐞₄ + 0.346 𝐞₅)

In [146]:
# only a single vector left in the dual of skew blades, 
# so it's the remaining eigenvector for the symmetric part
# with eigenvalue 1 or -1: identity or a reflection

skew_blades = bivector_split(ortho_curl/2, alg)
sym_vec = reduce(alg.ip, skew_blades, alg.pseudoscalar((1,)))
sym_vec = sym_vec / norm(sym_vec)
p = 0
if max_diff(ortho(sym_vec), -sym_vec) < 1e-6:
    p += 1

In [147]:
rotors = []
for blade in skew_blades:
    t = P(sum(alg.frame), blade)
    # blade_norm = norm(blade)
    # we can't simply rely on ortho_m:
    # angle = np.arcsin(blade_norm) has no unique solution in [0, pi].
    # (pi - angle) generates the same sine, but differnt cosine (alpha).
    
    alpha = np.median(terms_ratio(ortho(t) - (t | ortho_curl)/2, t))
    # half_blade_ortho = (angle/blade_norm/2) * blade.reverse()
    # rotors.append(blade_exp(half_blade_ortho))
    rotors.append(simple_rotor_sqrt(alpha + blade))
rotor = reduce(alg.gp, rotors)
if p == 1:
    rotor2 = sym_vec * rotor
else:
    rotor2 = rotor
(-1) ** p * rotor2.reverse().sw(x), ortho(x)

(0.481 𝐞₁ + 0.845 𝐞₂ + 0.465 𝐞₃ + 0.849 𝐞₄ + -0.0193 𝐞₅ + -1.84e-18 𝐞₁₂₃₄₅,
 0.481 𝐞₁ + 0.845 𝐞₂ + 0.465 𝐞₃ + 0.849 𝐞₄ + -0.0193 𝐞₅)

In [148]:
# On the other hand, we can solve the rotor part by Lasenby's method
frame = [item for blade in skew_blades for item in blade_split(blade, alg)]
r_frame = reciprocal(frame)
lasenby_rotor = sum(char_multi_linear(ortho, s, alg, frame, r_frame) for s in range(5))
terms_ratio(rotor, lasenby_rotor)

array([0.14474359, 0.14474359, 0.14474359, 0.14474359, 0.14474359,
       0.14474359, 0.14474359, 0.14474359, 0.14474359, 0.14474359,
       0.14474359, 0.14474359, 0.14474359, 0.14474359, 0.14474359,
       0.14474359])

In [151]:
# For orthogonal transform, we simply normalize it
l_rotor = lasenby_rotor/norm(lasenby_rotor)
if p == 1:
    l_rotor2 = l_rotor * sym_vec
else:
    l_rotor2 = l_rotor
(-1) ** p * l_rotor2.reverse().sw(x), ortho(x)

(0.481 𝐞₁ + 0.845 𝐞₂ + 0.465 𝐞₃ + 0.849 𝐞₄ + -0.0193 𝐞₅ + -1.11e-16 𝐞₁₂₃₄₅,
 0.481 𝐞₁ + 0.845 𝐞₂ + 0.465 𝐞₃ + 0.849 𝐞₄ + -0.0193 𝐞₅)

In [155]:
# Because sym_vec is orthogonal to rotor, we can extract sym_vec from l_rotor:
terms_ratio(l_rotor2.grade(1), sym_vec), terms_ratio(l_rotor | alg.pseudoscalar((1,)), sym_vec)

(array([0.43179804, 0.43179804, 0.43179804, 0.43179804, 0.43179804]),
 array([0.12442337, 0.12442337, 0.12442337, 0.12442337, 0.12442337]))

In [158]:
# From Graded Symmetry Groups: Plane and Simple, 
# instead of solving linear equations, the solutions are given algebraically
def tangent_split(R, alg: Algebra):  
    # FIXME: it produce nonsimple tangents
    m = alg.d // 2
    if m <= 1:
        return F
    Ck = [R.grade(2*r) for r in range(m+1)]
    Ck2 = [(-1)**(m - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(m+1)]
    Lk = np.roots(Ck2)
    tangents = []
    for l in Lk:
        even = sum(l ** r * c for r, c in enumerate(Ck[-1::-2]))
        odd = sum(l ** r * c for r, c in enumerate(Ck[-2::-2]))
        tangents.append(even * odd.reverse() / normsq(odd)) # FIXME: the inverse is wrong
    return tangents

In [163]:
tangents = tangent_split(l_rotor, alg)
rotors2 = []
for t in tangents:
    r = 1+t
    rotors2.append(r/norm(r))
rotors2, rotors 

([0.433 + -0.000851 𝐞₁₂ + -0.534 𝐞₁₃ + 0.289 𝐞₁₄ + 0.126 𝐞₁₅ + 0.459 𝐞₂₃ + -0.162 𝐞₂₄ + -0.184 𝐞₂₅ + 0.25 𝐞₃₄ + 0.267 𝐞₃₅ + -0.153 𝐞₄₅ + 4.7e-18 𝐞₁₂₃₄ + -1.88e-18 𝐞₁₂₃₅ + -7.53e-18 𝐞₁₂₄₅ + 7.64e-18 𝐞₁₃₄₅ + 4.59e-18 𝐞₂₃₄₅,
  0.99 + -0.00632 𝐞₁₂ + -0.00815 𝐞₁₃ + 0.025 𝐞₁₄ + -0.0213 𝐞₁₅ + 0.0279 𝐞₂₃ + 0.0695 𝐞₂₄ + -0.0713 𝐞₂₅ + 0.026 𝐞₃₄ + -0.0322 𝐞₃₅ + -0.0766 𝐞₄₅ + 1.08e-17 𝐞₁₂₃₄ + -4.3e-18 𝐞₁₂₃₅ + -1.72e-17 𝐞₁₂₄₅ + 1.75e-17 𝐞₁₃₄₅ + 1.05e-17 𝐞₂₃₄₅],
 [0.99 + -0.00642 𝐞₁₂ + 0.00304 𝐞₁₃ + 0.0193 𝐞₁₄ + -0.0244 𝐞₁₅ + 0.0187 𝐞₂₃ + 0.0742 𝐞₂₄ + -0.0687 𝐞₂₅ + 0.0212 𝐞₃₄ + -0.0385 𝐞₃₅ + -0.0748 𝐞₄₅,
  0.436 + 0.00477 𝐞₁₂ + -0.541 𝐞₁₃ + 0.274 𝐞₁₄ + 0.149 𝐞₁₅ + 0.446 𝐞₂₃ + -0.228 𝐞₂₄ + -0.125 𝐞₂₅ + 0.233 𝐞₃₄ + 0.302 𝐞₃₅ + -0.0891 𝐞₄₅])

In [165]:
gprod(rotors2), l_rotor, rotors2[0].grade(2) ** 2

(0.395 + -0.00358 𝐞₁₂ + -0.533 𝐞₁₃ + 0.297 𝐞₁₄ + 0.116 𝐞₁₅ + 0.466 𝐞₂₃ + -0.13 𝐞₂₄ + -0.213 𝐞₂₅ + 0.259 𝐞₃₄ + 0.25 𝐞₃₅ + -0.185 𝐞₄₅ + 0.0537 𝐞₁₂₃₄ + -0.0475 𝐞₁₂₃₅ + 0.0384 𝐞₁₂₄₅ + 0.0428 𝐞₁₃₄₅ + -0.0858 𝐞₂₃₄₅,
 0.432 + 0.00193 𝐞₁₂ + -0.535 𝐞₁₃ + 0.28 𝐞₁₄ + 0.137 𝐞₁₅ + 0.45 𝐞₂₃ + -0.193 𝐞₂₄ + -0.154 𝐞₂₅ + 0.24 𝐞₃₄ + 0.283 𝐞₃₅ + -0.121 𝐞₄₅ + 0.0532 𝐞₁₂₃₄ + -0.047 𝐞₁₂₃₅ + 0.038 𝐞₁₂₄₅ + 0.0424 𝐞₁₃₄₅ + -0.0849 𝐞₂₃₄₅,
 -0.813 + 1.04e-17 𝐞₁₂ + -3.47e-18 𝐞₁₄ + 6.94e-18 𝐞₁₅ + -1.39e-17 𝐞₂₅ + 0.0919 𝐞₁₂₃₄ + -0.0812 𝐞₁₂₃₅ + 0.0657 𝐞₁₂₄₅ + 0.0732 𝐞₁₃₄₅ + -0.147 𝐞₂₃₄₅)